In [31]:
import tensorflow as tf
import keras
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model, Model
from keras.layers import *
import os
import sys
import numpy as np
from score_dataset_helpers import *
import pandas as pd

In [2]:
base_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/data_dual_inputs/'
train_dir = base_dir + 'train'
validation_dir = base_dir + 'validation'
test_dir = base_dir + 'test'

## Define model

In [82]:
f1, f2, f3 = 8,16,32
k_size = (3,3)

pics_input = Input(shape = (75, 250, 3))
pics_c1 = Conv2D(filters = f1, kernel_size = k_size, activation = "relu")(pics_input)
pics_m1 = MaxPooling2D(k_size)(pics_c1)
pics_c2 = Conv2D(filters = f2, kernel_size = k_size, activation = "relu")(pics_m1)
pics_m2 = MaxPooling2D(k_size)(pics_c2)
pics_c3 = Conv2D(filters = f3, kernel_size = k_size, activation = "relu")(pics_m2)
pics_m3 = MaxPooling2D((2,2))(pics_c3)
pics_f = Flatten()(pics_m3)
pics_d = Dense(512, activation = "softmax")(pics_f)

nums_input = Input(shape = ([1]))
nums_features = Dense(1, activation = "softmax")(nums_input)

conc_layer = concatenate([pics_d, nums_features])
dense_1 = Dense(10, activation = "softmax")(conc_layer)

model = Model(inputs = [pics_input, nums_input], outputs = dense_1)
model.compile(loss = "categorical_crossentropy", optimizer = "sgd", metrics = ["accuracy"])

In [49]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 75, 250, 3)   0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 73, 248, 8)   224         input_5[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_7 (MaxPooling2D)  (None, 24, 82, 8)    0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 22, 80, 16)   1168        max_pooling2d_7[0][0]            
__________________________________________________________________________________________________
max_poolin

## Create vectors of scores from pictures

In [69]:
train_scores = train_dir + "/score"
train_pics = train_dir + "/pics"

#valid_scores_up = validation_dir + "/up/score"

#test_scores_up = test_dir + "/up/score"

In [70]:
train_names_scores = [f for f in os.listdir(train_scores)]
train_names_pics = [f for f in os.listdir(train_pics)]

#valid_names_up = [f for f in os.listdir(valid_scores_up)]

#test_names_up = [f for f in os.listdir(test_scores_up)]

In [71]:
print(str(len(train_names_scores)) + " training files for scores")
print(str(len(train_names_pics)) + " training files for pictures")

#print(str(len(valid_names_up)) + " validation files up")

#print(str(len(test_names_up)) + " test files up")

125 training files for scores
138training files for pictures


In [72]:
train_dataset_score = create_dataset("score", train_names_scores, base_dir + "train/")
#validation_dataset = create_dataset("validation/score", valid_names)
#test_dataset = create_dataset("test/score", test_names)

In [73]:
def get_result(result):
    resultstr = ''
    for i in range(5):
        resultstr += str(np.argmax(result[i]))
    return resultstr

In [74]:
score_model = load_model("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/models/score_v2.h5")

In [58]:
#Create vectors for scores and for labels
score_vec = []
for i in range(len(train_names)):
    y_pred = score_model.predict(train_dataset[i].reshape(1, 50, 160, 1))
    score_vec.append(get_result(y_pred))

label_vec = ['up' in x for x in train_names]

In [59]:
print_dat = pd.DataFrame({'score' : score_vec, 'label' : label_vec, 'orig_name' : train_names})
print_dat

,label,orig_name,score
0,False,right_1.png,00010
1,False,right_10.png,00050
2,False,right_11.png,00059
3,False,right_12.png,00067
4,False,right_13.png,00075
5,False,right_14.png,00082
6,False,right_15.png,00093
7,False,right_16.png,00097
8,False,right_18.png,00100
9,False,right_19.png,00125


In [76]:
x = load_img(train_pics + "/" + train_names[1], target_size = (75, 250))
x = img_to_array(x)
x.shape


(75, 250, 3)

In [77]:
pic_array = np.zeros((len(train_names_pics), 75, 250, 3))

for i in range(len(train_names_pics)):
    x = load_img(train_pics + '/' + train_names_pics[i], target_size = (75, 250))
    x = img_to_array(x)
    pic_array[i] = x
    
    

In [78]:
pic_array.shape

(138, 75, 250, 3)

In [81]:
model.fit([pic_array, score_vec], label_vec, epochs = 3)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:



#Remove same pictues for scores and pics (white, game over)???